# Install Necessary Libraries

In [ ]:
#already installed globaly
# !pip install mysqlalchemy
# !pip install pymysql

# 1. Establish the Database Connection using SQLAlchemy

In [24]:
from sqlalchemy import create_engine

#Define the database credentials
username = "root"
password = "root"
host = "localhost"
port = "3306"
database = "company"

#Construct the database connection URL
connection_string = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"

In [25]:
try:    
    #Create the SQLAlchemy engine
    db_conn = create_engine(connection_string)
    print(f"Connection to the {host} for user {username} created successful.")
except Exception as e:
    print(f"Connection could not be made due to the following error: \n", e)

Connection to the localhost for user root created successful.


# 2. Read Data from MySQL into a Pandas DataFrame:

- we can use use Pandas' `read_sql()` function to execute SQL queries and load the results directly into a DataFrame

In [26]:
import pandas as pd

# SQL query
sql_query = "SELECT * FROM employees"

# Read data into a Pandas DataFrame
# df_employees = pd.read_sql("SELECT * FROM employees", engine)
df_employees = pd.read_sql(sql_query, db_conn)
print(df_employees.head())

# Close the connection
db_conn.dispose()


   emp_Id emp_name  dept_Id   salary
0       1     John      2.0  75000.0
1       2     Jane      1.0  65000.0
2       3     Mike      2.0  80000.0
3       4    Emily      3.0  70000.0
4       5    Sarah      NaN  60000.0


In [27]:
# This code is good practice to write a code because of exception handling
# Read data into a Pandas DataFrame
try:
    df_employees = pd.read_sql(sql_query, db_conn)
    print(df_employees.head())  # Display the first few rows of the DataFrame
except Exception as e:
    print(f"Error reading from database: {e}")
finally:
    # It's good practice to close the connection
    db_conn.dispose()

   emp_Id emp_name  dept_Id   salary
0       1     John      2.0  75000.0
1       2     Jane      1.0  65000.0
2       3     Mike      2.0  80000.0
3       4    Emily      3.0  70000.0
4       5    Sarah      NaN  60000.0


# 3.  Write a Pandas DataFrame to a MySQL Table:

- to write data from a Pandas DataFrame to a MySQL table using the `to_sql()` method:
    - `name`: Specifies the name of the table in the MySQL database.
    - `con`: The SQLAlchemy connection engine.
    - `if_exists`: Determines how to handle the table if it already exists:
        - `'fail'`: Raises a ValueError.
        - `'replace'`: Drops the table and creates a new one.
        - `'append'`: Adds the new data to the existing table.
    - `index`: A boolean indicating whether to write the DataFrame index as a column in the SQL table. `False` is usually preferred.

In [ ]:
try:
    # Read data 
    df_departments = pd.read_sql("SELECT * from departments", db_conn)
    print("Data Read from MySQL:")
    print(df_departments.head())
    
    # performing join operation on employees and departments
    df_merged_emp_dep = pd.merge(df_employees, df_departments, on="dept_Id")
    
    # Write the modified DataFrame to a new table
    df_merged_emp_dep.to_sql("merged_employees_departments", db_conn, if_exists="replace", index=False)
    print(f"\nProcessed data written to table 'merged_employees_departments' in the database.")
    print(df_merged_emp_dep.head())
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    if 'db_conn' in locals():
        db_conn.dispose()

Data Read from MySQL:
   dept_Id        dept_name
0        1  Human Resources
1        2      Engineering
2        3        Marketing

Processed data written to table 'merged_employees_departments' in the database.
   emp_Id emp_name  dept_Id   salary        dept_name
0       1     John      2.0  75000.0      Engineering
1       2     Jane      1.0  65000.0  Human Resources
2       3     Mike      2.0  80000.0      Engineering
3       4    Emily      3.0  70000.0        Marketing
